In [ ]:
!pip install transformers
!pip install pandas

In [ ]:
#import libraries
import pandas as pd
from transformers import pipeline
import time

In [ ]:
from google.colab import drive
drive.mount ('/drive')

Mounted at /drive


In [ ]:
df = pd.read_csv('/content/Cleaned Sheet of Grouped Questions - Sheet1 (1).csv' )
job_role= df['Job Role']
questions= df['Questions']
answers= df['Answers']
rating= df['Rating']


In [ ]:
df.isnull().sum()

Group             0
Topic             0
Job Role          0
Questions         0
Collaborator      0
Sources         796
Rating            2
Answers           2
dtype: int64

In [ ]:
df.dropna(subset=['Answers'], inplace=True)
df.dropna(subset=['Rating'], inplace=True)
df.isnull().sum()

Group             0
Topic             0
Job Role          0
Questions         0
Collaborator      0
Sources         793
Rating            0
Answers           0
dtype: int64

In [ ]:
#oneshortclassification pipeline
classifier = pipeline ("zero-shot-classification", model= 'facebook/bart-large-mnli')

In [ ]:
start_time = time.time()

#labeling type categories
class_label = ["Behavioral","Communication","Introduction","Conclusion","Technical"]

#creating empty dict for easy coversion to dataframe
d = {"Job Role":[], 'Questions': [], 'Introduction': [], 'Behavioral': [], 'Communication': [],"Conclusion":[],"Technical":[],"Category": [],"Answers":[], "Ratings":[]}

# Initialize variables to track the category with the highest percentage
# max_category = None
# max_percentage = -1

for idx, question in enumerate(questions):
    max_category = None
    max_percentage = -1
    result = classifier(question, class_label)

    d['Questions'].append(result['sequence'])
    d['Answers'].append(answers[idx])  # Append corresponding answer
    d['Ratings'].append(rating[idx])
    d['Job Role'].append(job_role[idx])
    for label, score in zip(result["labels"], result["scores"]):
        percentage = int(score * 100)
        d[label].append(str(percentage) + '%')

       # Update max_category if the current percentage is higher
        if percentage > max_percentage:
            max_category = label
            max_percentage = percentage

    # Add the category with the highest percentage to the 'Category' column
    d['Category'].append(max_category)


output_df = pd.DataFrame(d)
print (output_df)


end_time = time.time()
print("Execution time : ", end_time - start_time, ' secondes', '----', (end_time - start_time)/60, ' minutes')


               Job Role                                         Questions  \
0                 Nurse         why did you decide on a career as a nurse   
1                 Nurse         why did you decide on a career as a nurse   
2                 Nurse         why did you decide on a career as a nurse   
3                 Nurse         why did you decide on a career as a nurse   
4                 Nurse         why did you decide on a career as a nurse   
...                 ...                                               ...   
1069  Medical Assistant  what do you think makes a good medical assistant   
1070            Nursing                   what makes you a good candidate   
1071            Nursing                   what makes you a good candidate   
1072            Nursing                   what makes you a good candidate   
1073            Nursing                   what makes you a good candidate   

     Introduction Behavioral Communication Conclusion Technical      Catego

In [ ]:
output_df['Category'].value_counts()

Introduction     301
Communication    268
Conclusion       247
Behavioral       192
Technical         66
Name: Category, dtype: int64

In [ ]:
output_df['Answers']=output_df['Answers'].str.replace("â€™", "'")
output_df

,Job Role,Questions,Introduction,Behavioral,Communication,Conclusion,Technical,Category,Answers,Ratings
0,Nurse,why did you decide on a career as a nurse,34%,33%,4%,22%,5%,Introduction,I come from a long line of nurses. Both my gra...,Good
1,Nurse,why did you decide on a career as a nurse,34%,33%,4%,22%,5%,Introduction,I want to be a nurse because I've always been ...,Average
2,Nurse,why did you decide on a career as a nurse,34%,33%,4%,22%,5%,Introduction,I was drawn to nursing because of the diverse ...,Good
3,Nurse,why did you decide on a career as a nurse,34%,33%,4%,22%,5%,Introduction,During my time as a volunteer at a local hospi...,Good
4,Nurse,why did you decide on a career as a nurse,34%,33%,4%,22%,5%,Introduction,I have always admired the resilience and adapt...,Good
...,...,...,...,...,...,...,...,...,...,...
1069,Medical Assistant,what do you think makes a good medical assistant,25%,24%,10%,24%,16%,Introduction,My previous job working as a assistant provide...,good
1070,Nursing,what makes you a good candidate,32%,17%,11%,25%,13%,Introduction,Personality Communication skills Drive and det...,poor
1071,Nursing,what makes you a good candidate,32%,17%,11%,25%,13%,Introduction,I've been interested in the medical field sinc...,poor
1072,Nursing,what makes you a good candidate,32%,17%,11%,25%,13%,Introduction,I'm proud of my ability to provide the best po...,good


In [ ]:
data=output_df.drop(['Introduction', 'Behavioral', 'Communication','Conclusion','Technical'], axis=1)
data

,Job Role,Questions,Category,Answers,Ratings
0,Nurse,why did you decide on a career as a nurse,Introduction,I come from a long line of nurses. Both my gra...,Good
1,Nurse,why did you decide on a career as a nurse,Introduction,I want to be a nurse because I've always been ...,Average
2,Nurse,why did you decide on a career as a nurse,Introduction,I was drawn to nursing because of the diverse ...,Good
3,Nurse,why did you decide on a career as a nurse,Introduction,During my time as a volunteer at a local hospi...,Good
4,Nurse,why did you decide on a career as a nurse,Introduction,I have always admired the resilience and adapt...,Good
...,...,...,...,...,...
1069,Medical Assistant,what do you think makes a good medical assistant,Introduction,My previous job working as a assistant provide...,good
1070,Nursing,what makes you a good candidate,Introduction,Personality Communication skills Drive and det...,poor
1071,Nursing,what makes you a good candidate,Introduction,I've been interested in the medical field sinc...,poor
1072,Nursing,what makes you a good candidate,Introduction,I'm proud of my ability to provide the best po...,good


In [ ]:
data.to_csv(r'C:\Users\Downloads\final.csv')